# Hedonic Clustering
> _Finding Clusters with Cooperative Game Theory._<br/>
> A research experiment of _Daniel Sadoc_$^\ast$ and _Lucas Lopes_$^\ast$.<br/>
> $^\ast$Federal University of Rio de Janeiro.<br/>
> December, 2018.

## Parameters
For convenience and reachability, let's put the parameters on top.<br/>
To understand each parameter, click: [dataset](#converting-a-csv-file-to-a-python-dictionary), [initial](#creating-the-initial-condicions), [verbose](#printing-the-algorithm-steps), [alpha](#-main-function)

In [1]:
dataset = "datasets/samples/sample_2.csv"
initial = ['e']
verbose = True
alpha   = 0.95

## Main Function

The hedonic function tells how good a node is 'fit' in a given network.
It takes:

1. A $\alpha$ value, where $\alpha \in [0,1]$ (_closer to 1 is broader_);
2. The number of vertices in a particular graph;
3. The number of connections that a node (in this graph) has.

And returns the _"score"_ of that node.

In [2]:
def hedonic(alpha, num_vert, my_connections):
    a = (1 - alpha) *             my_connections
    b =      alpha  * (num_vert - my_connections - 1)
    return a - b

## Helper Functions

### Converting a CSV file to a Python Dictionary

Since the graph network is represented as a dictonary in the form$^\ast$, and usually a dataset that stores a network is in the _.csv_ format, it is helpful to have a function that convert a dataset in a form that the algorithm can read and operate on it.

To do so, the _.csv_ file must be in the following way:

| From Node | To Node  |
| :-------: | :------: |
| _number_  | _number_ |

$^\ast$**Key** is the vertices; and **Value** is a list of its connections.

In [3]:
import csv

def insert(d, a, b):
    if a not in d:
        d[a] = [b]
    elif b not in d[a]:
        d[a].append(b)
    return d

def csv_2_dict(file):
    d = {}
    with open(file, 'r') as f:
        table = csv.reader(f)
        for row in table:
            a = int(row[0])
            b = int(row[1])
            d = insert(d, a, b)
            d = insert(d, b, a)
    return d

### Doing graph operations in a Dictionary

It is possible to do many operations in a graph, such as:

- Add a new vertice;
- Remove an old one;
- Move a vertice from a graph to another.

So here are helper functions to do this above operations. But before that, we need to import the _copy_ library because of this$^\ast$.

$^\ast$`dict.copy()` method do a [shallow copy](https://docs.python.org/2/library/stdtypes.html#dict.copy). We need a [deep copy](https://docs.python.org/2/library/copy.html#copy.deepcopy) to avoid [this problem](https://stackoverflow.com/questions/3975376/understanding-dict-copy-shallow-or-deep).

In [4]:
import copy

def add(vert, original):
    g = copy.deepcopy(original)
    g[vert] = []
    for key in g:
        if vert in graph[key]:
            g[key].append(vert)
            g[vert].append(key)
    return g

def remove(vert, original):
    g = copy.deepcopy(original)
    for v in g[vert]:
        g[v].remove(vert)
    g.pop(vert)
    return g

def move(verts, from_g, to_g):
    if type(verts) is list:
        for v in verts:
            from_g = remove(v, from_g)
            to_g   = add(v, to_g)
    else:
        from_g = remove(verts, from_g)
        to_g   = add(verts, to_g)
    return from_g, to_g

### Creating the initial condicions

We have 2 graphs, one is the _cluster_ that we're looking for, e the other one is the original graph without those nodes in the cluster (a.k.a _remain_).

These 2 graphs (_cluster_ and _remain_) could be initiate in different ways, varying by answer this questions:

1. How many nodes?
2. It will be manually selected or random choices?

To do this different combinations, just change the value of the variable `init`.<br/>
There are 3 modes and its respectives parameters:

1. **Random mode**<br/>
    - Select a specific number of vertices randomly.<br/>
    - `['r', num]` where `num` is the amount that will be selected.


2. **Select mode**<br/>
    - Inform the specific vertices that you want to initiate.<br/>
    - `['s', num]` where `num` could be just the # of vertice or a list of them.
    
    
3. **Empty mode**<br/>
    - Initiate with no vertices.<br/>
    - `['e']` (It is the same that initiate with all node, just changes the reference)

In [5]:
import random

def rand(amount):
    v = []
    i = 0
    if amount < 0:
        amount += len(graph)
    while i < amount:
        r = random.choice(list(graph.keys()))
        if r not in v:
            v.append(r)
            i += 1
    return v

def init(option):
    
    if option[0] == 's':
        return option[1]
    
    if option[0] == 'r':
        return rand(option[1])
    
    if option[0] == 'e':
        return []

### Printing the algorithm steps

If the verbose mode is activate (_i.e._ `verbose == True`) it will print, at each iteration, this informations:

- The nodes and its hedonic value for move and stay where they are;
- Which nodes are in the _cluster_ and which is in _remain_;
- Who will move and what is its alpha.

In [6]:
def print_desire(n, m, s):
    print('node:', n, '| move: {:.2f} | stay: {:.2f}'.format(m, s))

def print_node(w):
    print('\n-> will move:', w[0], '| its alpha: {:.2f}'.format(w[1]))
    
separator  = '\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n'
graph_diff = '----- /cluster\ ---- \\remain/ -----'

## Learning Funcions

### Tells by how much a node wants to move

- It receives a node, its graph and the other graph which will be compare to;
- Calculate the hedonic value of been here, and if it chose to move;
- If the move desire is greater or equal$^\ast$ than staying here, the function returns its respective hedonic value;
- Else it return _negative inifinity_, because we only care about who want to move.

$^\ast$Let's consider a draw as _move_, because if has someone with a greater move desire, it will be given priority to it.

In [7]:
def move_desire(node, here, other):
    there = add(node, other)
    stay  = hedonic(alpha, len(here), len(here[node]))
    move  = hedonic(alpha, len(there), len(there[node]))
    
    if verbose: print_desire(node, move, stay)
    
    if move >= stay:
        return move
    else:
        return float("-inf")

### Who has the greater move desire?

- It receives the node with the greater move desire so far, the reference graph and the other one;
- Ask for wach node its desire to move;
- If its desire is greater than the current chosen one, it'll be the new chosen node.

In [8]:
def who_will_move(chosen_node, reference_g, other_g):

    for key in reference_g:
        node_desire = move_desire(key, reference_g, other_g)
        
        if node_desire > chosen_node[1]:
            chosen_node = [key, node_desire]
    
    return chosen_node

### Convergence

- It receives both _cluster_ and _remain_ graphs;
- Iterate to each of them to know who will move in the next iteration;
- If there's no one who want to move, the algorthm converged and return the 2 graphs.

In [9]:
def converge(remain, cluster):
    
    wanna_move = True
    while wanna_move:

        if verbose:   print(separator)
        chosen_node = [None, float("-inf")]
        chosen_node = who_will_move(chosen_node, cluster, remain)        
        if verbose:   print(graph_diff)
        chosen_node = who_will_move(chosen_node, remain, cluster)
        if verbose:   print_node(chosen_node)
        
        if chosen_node[0]:
            if chosen_node[0] in cluster:
                cluster, remain = move(chosen_node[0], cluster, remain)
            else:
                remain, cluster = move(chosen_node[0], remain, cluster)
        else:
            wanna_move = False

    return remain, cluster

### Set the original graph and initial condicions

- Convert the dataset that was selected to a dictonary;
- Read the initiate option.

In [10]:
graph = csv_2_dict(dataset)
init  = init(initial)

### Register the begin time

- Get the current time just before the training begin.

In [11]:
from datetime import datetime
begin = datetime.now()

### Training!

- Create the _cluster_ and _remain_ graphs;
- Seek for convegence.

In [12]:
remain, cluster = move(init, graph, {})
remain, cluster = converge(remain, cluster)


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

----- /cluster\ ---- \remain/ -----
node: 1 | move: 0.00 | stay: -5.65
node: 3 | move: 0.00 | stay: -2.65
node: 2 | move: 0.00 | stay: -5.65
node: 4 | move: 0.00 | stay: -2.65
node: 5 | move: 0.00 | stay: -2.65
node: 6 | move: 0.00 | stay: -2.65
node: 7 | move: 0.00 | stay: -4.65
node: 8 | move: 0.00 | stay: -4.65

-> will move: 1 | its alpha: 0.00

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

node: 1 | move: -5.65 | stay: 0.00
----- /cluster\ ---- \remain/ -----
node: 3 | move: 0.05 | stay: -2.70
node: 2 | move: -0.95 | stay: -4.70
node: 4 | move: -0.95 | stay: -1.70
node: 5 | move: -0.95 | stay: -1.70
node: 6 | move: -0.95 | stay: -1.70
node: 7 | move: -0.95 | stay: -3.70
node: 8 | move: -0.95 | stay: -3.70

-> will move: 3 | its alpha: 0.05

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

node: 1 | move: -5.70 | stay: 0.05
node: 3 | move: -2.70 | stay: 0.05
----- /cluster\ ---- \remain/ -----
node: 2 | move: -1.90 | stay: -3.75
node: 4 | move: -0.90 | stay: -1.75


### How long did it takes?

- Print the diffent between the time when the training started and finished.

In [13]:
done = datetime.now()
print('Converge in:', done - begin)

Converge in: 0:00:00.046186


## Results

### Showing the graphs

- Print the original graph, the result cluster, and the remain.

In [14]:
if verbose:
    print('\ngraph:\n', graph)
    print('\ncluster:\n', cluster)
    print('\nremain:\n', remain)


graph:
 {1: [3], 3: [1, 4, 5, 6], 2: [4], 4: [2, 3, 5, 6], 5: [3, 4, 6, 7], 6: [3, 4, 5, 8], 7: [5, 8], 8: [6, 7]}

cluster:
 {1: [3], 3: [1, 4, 5, 6], 4: [3, 5, 6], 5: [3, 4, 6], 6: [3, 4, 5]}

remain:
 {2: [], 7: [8], 8: [7]}


### Export cluster

- Export the resulting _cluster_ dictionary into a JSON file.

In [15]:
import json
 
j = json.dumps(cluster)
f = open("cluster.json","w")
f.write(j)
f.close()